In [0]:
%sql
SELECT * FROM `walmart`.`default`.`walmart_sales_data_csv`
limit 5;

Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,2024-03-20T13:08:00Z,Ewallet,522.83,4.761904762,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.82,80.22,2019-03-08,2024-03-20T10:29:00Z,Cash,76.4,4.761904762,3.82,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,2024-03-20T13:23:00Z,Credit card,324.31,4.761904762,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.288,489.048,2019-01-27,2024-03-20T20:33:00Z,Ewallet,465.76,4.761904762,23.288,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,2024-03-20T10:37:00Z,Ewallet,604.17,4.761904762,30.2085,5.3


# Feature Engineering

### Add Time of day Column

In [0]:
%sql
SELECT Time,
(CASE
		WHEN `time` BETWEEN "00:00:00" AND "12:00:00" THEN "Morning"
        WHEN `time` BETWEEN "12:01:00" AND "16:00:00" THEN "Afternoon"
        ELSE "Evening"
    END) AS time_of_day

FROM `walmart`.`default`.`walmart_sales_data_csv`
 limit 5
;

Time,time_of_day
2024-03-20T13:08:00Z,Afternoon
2024-03-20T10:29:00Z,Morning
2024-03-20T13:23:00Z,Afternoon
2024-03-20T20:33:00Z,Evening
2024-03-20T10:37:00Z,Morning


In [0]:
%sql
ALTER TABLE `walmart`.`default`.`walmart_sales_data_csv`
ADD COLUMN time_of_day VARCHAR(20);


In [0]:
%sql
SELECT time_of_day FROM  `walmart`.`default`.`walmart_sales_data_csv`
limit 5;

time_of_day
null
null
null
null
null


In [0]:
%sql

update  `walmart`.`default`.`walmart_sales_data_csv`
set time_of_day=
(
  CASE
		WHEN `time` BETWEEN "00:00:00" AND "12:00:00" THEN "Morning"
        WHEN `time` BETWEEN "12:01:00" AND "16:00:00" THEN "Afternoon"
        ELSE "Evening"
    END
);

num_affected_rows
1000


In [0]:
%sql
SELECT time_of_day FROM  `walmart`.`default`.`walmart_sales_data_csv`
limit 5;

time_of_day
Afternoon
Morning
Afternoon
Evening
Morning


### Add Day name Column

In [0]:
%sql
select date,
      dayofweek(date)
FROM `walmart`.`default`.`walmart_sales_data_csv` 
limit 5;

date,dayofweek(date)
2019-01-05,7
2019-03-08,6
2019-03-03,1
2019-01-27,1
2019-02-08,6


In [0]:
# %sql

# update `walmartsales_dataanalysis`.`default`.`walmart_sales_data_csv`
# SET day_number = DAYOFWEEK(date);

In [0]:
%sql

select date,
    DATE_FORMAT(date , 'MM')

FROM `walmart`.`default`.`walmart_sales_data_csv`
limit 5;

date,"date_format(date, MM)"
2019-01-05,01
2019-03-08,03
2019-03-03,03
2019-01-27,01
2019-02-08,02


# Business Questions to Ask

In [0]:
%sql
-- Q1 How many unique cities does the data have?

select distinct city 
from  `walmart`.`default`.`walmart_sales_data_csv`

city
Naypyitaw
Mandalay
Yangon


In [0]:
%sql
-- In which city is each branch?
SELECT 
	DISTINCT city,
    branch
from  `walmart`.`default`.`walmart_sales_data_csv`

city,branch
Naypyitaw,C
Yangon,A
Mandalay,B


In [0]:
%sql
-- How many unique product lines does the data have?
SELECT
	DISTINCT `product line`
from  `walmart`.`default`.`walmart_sales_data_csv`


product line
Home and lifestyle
Fashion accessories
Health and beauty
Electronic accessories
Food and beverages
Sports and travel


In [0]:
%sql
-- What is the most selling product line 
SELECT
	sum(Quantity)  as qty,
  `Product line`
  
from  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY 2 
ORDER BY 1 DESC;

qty,Product line
971,Electronic accessories
952,Food and beverages
920,Sports and travel
911,Home and lifestyle
902,Fashion accessories
854,Health and beauty


In [0]:
%sql
-- What product line had the largest revenue?
SELECT
	`product line`,
	SUM(total) as `total revenue`
from  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY 1
ORDER BY 2 DESC;

product line,total revenue
Food and beverages,56144.844000000005
Sports and travel,55122.826499999996
Electronic accessories,54337.531500000005
Fashion accessories,54305.895
Home and lifestyle,53861.91300000001
Health and beauty,49193.739000000016


In [0]:
%sql
-- What is the city with the largest revenue?
SELECT
	branch,
	city,
	ROUND(SUM(total), 2) AS total_revenue
from  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY city, branch 
ORDER BY total_revenue DESC;


branch,city,total_revenue
C,Naypyitaw,110568.71
A,Yangon,106200.37
B,Mandalay,106197.67


In [0]:
%sql
-- What product line had the largest VAT?
SELECT
	`product line`,
	ROUND(SUM(`Tax 5%`), 2) as avg_tax
from  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `product line`
ORDER BY avg_tax DESC;


product line,avg_tax
Food and beverages,2673.56
Sports and travel,2624.9
Electronic accessories,2587.5
Fashion accessories,2586.0
Home and lifestyle,2564.85
Health and beauty,2342.56


In [0]:
%sql
-- Fetch each product line and add a column to those product 
-- line showing "Good", "Bad". Good if its greater than average sales

SELECT 
	AVG(quantity) AS avg_qnty
from  `walmart`.`default`.`walmart_sales_data_csv`;

SELECT
	`product line`,
	CASE
		WHEN AVG(quantity) > 6 THEN "Good"
        ELSE "Bad"
    END AS remark
from  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `product line`;

product line,remark
Home and lifestyle,Bad
Fashion accessories,Bad
Health and beauty,Bad
Electronic accessories,Bad
Food and beverages,Bad
Sports and travel,Bad


In [0]:
%sql
-- what is the most common payment method 
select distinct Payment, count(Payment) as `Number of transactions` from `walmartsales_dataanalysis`.`default`.`walmart_sales_data_csv` 
group by 1 

Payment,Number of transactions
Ewallet,345
Cash,344
Credit card,311


In [0]:
%sql
-- Which branch sold more products than average product sold?
select 
	branch, 
    SUM(quantity) AS qnty
from  `walmart`.`default`.`walmart_sales_data_csv`
group by branch
having SUM(quantity) > (select AVG(quantity)  from  `walmart`.`default`.`walmart_sales_data_csv` )
order by 2 desc;

branch,qnty
A,1859
C,1831
B,1820


In [0]:
%sql

-- What is the most common product line by gender
SELECT
	gender,
    `product line`,
    COUNT(gender) AS total_cnt
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY gender, `product line`

ORDER BY total_cnt DESC;

gender,product line,total_cnt
Female,Fashion accessories,96
Female,Food and beverages,90
Female,Sports and travel,88
Male,Health and beauty,88
Male,Electronic accessories,86
Male,Food and beverages,84
Female,Electronic accessories,84
Male,Fashion accessories,82
Male,Home and lifestyle,81
Female,Home and lifestyle,79


In [0]:
%sql
-- What is the average rating of each product line
SELECT
	ROUND(AVG(rating), 2) as avg_rating,
    `product line`
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `product line`
ORDER BY avg_rating DESC;

avg_rating,product line
7.11,Food and beverages
7.03,Fashion accessories
7.0,Health and beauty
6.92,Electronic accessories
6.92,Sports and travel
6.84,Home and lifestyle



# Customers Analysis Questions 

In [0]:
%sql
-- How many unique customer types does the data have?
SELECT
	DISTINCT `customer type`
FROM  `walmart`.`default`.`walmart_sales_data_csv`;


customer type
Member
Normal


In [0]:
%sql

-- How many unique payment methods does the data have?
SELECT
	DISTINCT payment
FROM  `walmart`.`default`.`walmart_sales_data_csv`;


payment
Ewallet
Cash
Credit card


In [0]:
%sql

-- What is the most common customer type?
SELECT
	`customer type`,
	count(*) as count
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `customer type`
ORDER BY count DESC;

customer type,count
Member,501
Normal,499


In [0]:
%sql
-- What is the gender of most of the customers?
SELECT
	gender,
	COUNT(*) as gender_cnt
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY gender
ORDER BY gender_cnt DESC;

gender,gender_cnt
Female,501
Male,499


In [0]:
%sql

-- What is the gender distribution per branch?
SELECT
	gender,
	COUNT(*) as gender_cnt
FROM  `walmart`.`default`.`walmart_sales_data_csv`
WHERE branch = "C"
GROUP BY gender
ORDER BY gender_cnt DESC;

gender,gender_cnt
Female,178
Male,150


In [0]:
%sql

-- Which time of the day do customers give most ratings?
SELECT
	time_of_day,
	AVG(rating) AS avg_rating
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY time_of_day
ORDER BY avg_rating DESC;

time_of_day,avg_rating
Afternoon,7.031299734748012
Morning,6.960732984293193
Evening,6.926851851851853



# Sales Analysis Questions 

In [0]:
%sql
-- Which of the customer types brings the most revenue?
SELECT
	`Customer type`,
	SUM(total) AS total_revenue
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `Customer type`
ORDER BY total_revenue;


Customer type,total_revenue
Normal,158743.30500000005
Member,164223.44400000002


In [0]:
%sql
-- Which city has the largest tax/VAT percent?
SELECT
	city,
    ROUND(AVG(`Tax 5%`), 2) AS avg_tax_pct
FROM  `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY city 
ORDER BY avg_tax_pct DESC;

city,avg_tax_pct
Naypyitaw,16.05
Mandalay,15.23
Yangon,14.87


In [0]:
%sql
-- Which customer type pays the most in VAT?
SELECT
	`Customer type`,
	AVG(`Tax 5%`) AS total_tax
FROM `walmart`.`default`.`walmart_sales_data_csv`
GROUP BY `Customer type`
ORDER BY total_tax;


Customer type,total_tax
Normal,15.148707414829664
Member,15.609109780439125
